# Importing the Libraries

In [1]:
Team Id: PNT2022TMID45551

SyntaxError: invalid syntax (1994840342.py, line 1)

In [2]:
Global sales data analytics with an Interactive Dashboard

SyntaxError: invalid syntax (2925259220.py, line 1)

In [3]:
Dataset used: https://www.kaggle.com/apoorvaappz/global-super-store-dataset

SyntaxError: invalid syntax (1101637488.py, line 1)

In [4]:
# data operation libraries
import numpy as np
import pandas as pd

# importing visualisation libraries
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# for chloroplath plotting
import chart_studio.plotly as py
import plotly.graph_objs as go 
import plotly
import cufflinks as cf
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
cf.go_offline()

# for datetime operations
import datetime as dt

# pandas general settings
pd.options.display.max_columns = None

ModuleNotFoundError: No module named 'chart_studio'

# Importing the Dataset

In [ ]:
data = pd.read_csv('../input/global-super-store-dataset/Global_Superstore2.csv', encoding='windows-1252')

# Data Preparation

In [ ]:
data.head(2) #taking a look at the dataframe structure

In [ ]:
# correcting 'Order Date' variable
data[['order_day','order_month','order_year']] = data['Order Date'].str.split('-', expand=True)
data['Order Date'] = data['order_year'] + '/' + data['order_month'] + '/' + data['order_day']
data['Order Date'] = pd.to_datetime(data['Order Date'])

In [ ]:
# doing likewise for 'Ship Date'
data[['ship_day','ship_month','ship_year']] = data['Ship Date'].str.split('-', expand=True)
data['Ship Date'] = data['ship_year'] + '/' + data['ship_month'] + '/' + data['ship_day']
data['Ship Date'] = pd.to_datetime(data['Ship Date'])

In [ ]:
# dropping the support columns
data.drop(columns=['order_day','order_month','order_year','ship_day','ship_month','ship_year'], inplace=True)

In [ ]:
data.info() #checkout the data types/ null rows and memory consumption

In [ ]:
# let's check out the columns which are suitable category column type

data.nunique()

In [ ]:
data['Ship Mode'] = data['Ship Mode'].astype('category')
data['Segment'] = data['Segment'].astype('category')
data['Country'] = data['Country'].astype('category')
data['Market'] = data['Market'].astype('category')
data['Region'] = data['Region'].astype('category')
data['Category'] = data['Category'].astype('category')
data['Sub-Category'] = data['Sub-Category'].astype('category')
data['Order Priority'] = data['Order Priority'].astype('category')

In [ ]:
data.info() #check the reduction in memory consumption

In [ ]:
# making sure neither of our category columns have leading spaces

def remove_leading_spaces(df):
    for cols in df.columns:
        if df[cols].dtypes in ['object','category']:
            df[cols] = df[cols].str.strip()
        return df

In [ ]:
data = remove_leading_spaces(data)

In [ ]:
data.head(2)

In [ ]:
# generating years from our 'Order_year' variable because we are going 
# to need this in future analysis

data['Order_year'] = data['Order Date'].dt.year

In [ ]:
# also total unique customer count is something we need in our future analysis

print('Number of unique customers made purchase in 2011: {}'.format(data[data['Order_year']==2011]['Customer Name'].nunique()))
print('Number of unique customers made purchase in 2012: {}'.format(data[data['Order_year']==2012]['Customer Name'].nunique()))
print('Number of unique customers made purchase in 2013: {}'.format(data[data['Order_year']==2013]['Customer Name'].nunique()))
print('Number of unique customers made purchase in 2014: {}'.format(data[data['Order_year']==2014]['Customer Name'].nunique()))

In [ ]:
def total_purchase_in_year(row):
    Order_year = row[24]
    
    if Order_year in [2011,2012,2013]:
        return 795
    else:
        return 794
    
    
# generating  'unique_customers_within_year' based on associated year value
# for that particular row

data['unique_customers_within_year'] = data.apply(total_purchase_in_year, axis='columns')

In [ ]:
# Generating 'Revenue' column
data['Revenue'] = data['Sales'] * data['Quantity']

In [ ]:
purchase_frequency = data.groupby(['Order_year','Customer Name'])

In [ ]:
purchase_frequency.agg({'Customer Name': 'count',
                       'unique_customers_within_year': 'min',
                       'Revenue': 'sum',
                       'Profit': 'sum'}) 

In [ ]:
analysis_result = purchase_frequency.agg({'Customer Name': 'count',
                       'unique_customers_within_year': 'min',
                       'Revenue': 'sum',
                       'Profit': 'sum'})

In [ ]:
analysis_result.rename(mapper={'Customer Name': 'Purchase_during_year'}, axis=1, inplace=True)

In [ ]:
analysis_result['Customer_purchase_frequency'] = analysis_result['Purchase_during_year']/analysis_result['unique_customers_within_year'] *100

In [ ]:
analysis_result.head(5)

In [ ]:
tmp_df = analysis_result.reset_index()

In [ ]:
tmp_df.head()

In [ ]:
grouped_object = tmp_df.groupby(['Order_year'])

In [ ]:
freq_df = pd.DataFrame(columns=tmp_df.columns)

In [ ]:
for g,d in grouped_object:
    highest_freq_customers = d.nlargest(1, 'Customer_purchase_frequency')
    freq_df = pd.concat([freq_df, highest_freq_customers])

In [ ]:
def highlight_cols(x): 
    df = x.copy()
    df.loc[:, ['Customer Name','Customer_purchase_frequency']] = 'background-color: green'
    df[['Order_year','Purchase_during_year','unique_customers_within_year','Revenue','Profit']] = 'background-color: grey'
    return df 

In [ ]:
display(freq_df.style.apply(highlight_cols, axis = None))

In [ ]:
rev_df = pd.DataFrame(columns=tmp_df.columns)

In [ ]:
for g,d in grouped_object:
    highest_rev_customers = d.nlargest(1, 'Revenue')
    rev_df = pd.concat([rev_df, highest_rev_customers])

In [ ]:
def highlight_cols(x): 
    df = x.copy()
    df.loc[:, ['Customer Name','Revenue']] = 'background-color: green'
    df[['Order_year','Purchase_during_year','unique_customers_within_year','Profit','Customer_purchase_frequency']] = 'background-color: grey'
    return df 

In [ ]:
display(rev_df.style.apply(highlight_cols, axis = None))

In [ ]:
profit_df = pd.DataFrame(columns=tmp_df.columns)

In [ ]:
for g, d in grouped_object:
    highest_profit = d.nlargest(1, 'Profit')
    profit_df = pd.concat([profit_df,highest_profit])

In [ ]:
def highlight_cols(x): 
    df = x.copy()
    df.loc[:, ['Customer Name','Profit']] = 'background-color: green'
    df[['Order_year','Purchase_during_year','unique_customers_within_year','Revenue','Customer_purchase_frequency']] = 'background-color: grey'
    return df 

In [ ]:
display(profit_df.style.apply(highlight_cols, axis = None))

In [5]:
segment_group = data.groupby(['Order_year','Segment'])

NameError: name 'data' is not defined

In [6]:
high_profit_df = segment_group.agg({'Profit':'sum'}).unstack()
high_profit_df.style.background_gradient(cmap='Spectral', subset=pd.IndexSlice[:, pd.IndexSlice[:,'Consumer']])

NameError: name 'segment_group' is not defined

In [7]:
country_group = data.groupby(['Country'])

NameError: name 'data' is not defined

In [8]:
customer_distribution = country_group.agg({'Customer ID':'count'})
customer_distribution.columns = ['Customer_count']
customer_distribution.reset_index(inplace=True)
customer_distribution

NameError: name 'country_group' is not defined

In [9]:
country_map = dict(type='choropleth',
           locations=customer_distribution['Country'],
           locationmode='country names',
           z=customer_distribution['Customer_count'],
            reversescale = True,
           text=customer_distribution['Country'],
           colorscale='earth',
           colorbar={'title':'Customer Count'})

NameError: name 'customer_distribution' is not defined

In [10]:
layout = dict(title='Customer Distribution over Countries',
             geo=dict(showframe=False,projection={'type':'mercator'}))

In [11]:
choromap = go.Figure(data = [country_map],layout = layout)
iplot(choromap)

NameError: name 'go' is not defined

In [12]:
country_group = data.groupby('Country')

NameError: name 'data' is not defined

In [13]:
import squarify

In [14]:
top_10_sales = country_sales.nlargest(10, 'Sales')
top_10_sales.index

NameError: name 'country_sales' is not defined

In [15]:
plt.figure(figsize=(15,7))
revs = top_10_sales['Sales'].values
labels = ['United States: 2297200.8603',
         'Australia: 925235.853',
         'France: 858931.083',
         'China: 700562.025',
         'Germany: 628840.0305',
         'Mexico: 622590.61752',
         'India: 589650.105',
         'United Kingdom: 528576.3',
         'Indonesia: 404887.4979',
         'Brazil: 361106.41896']
squarify.plot(revs, label=labels,color= sns.color_palette('copper'), alpha=0.7)
plt.show()

NameError: name 'top_10_sales' is not defined

<Figure size 1080x504 with 0 Axes>

In [ ]:
year_category_group = data.groupby(['Order_year','Sub-Category'])

In [16]:
year_category_proft_df = year_category_group.agg({'Profit':'sum'})
year_category_proft_df

NameError: name 'year_category_group' is not defined

In [17]:
year_category_proft_df.reset_index(inplace=True)
category_yearly_profit = year_category_proft_df.groupby('Order_year')
top5_profit_category = pd.DataFrame(columns=year_category_proft_df.columns)

NameError: name 'year_category_proft_df' is not defined

In [18]:
for g, d in category_yearly_profit:
    high_profit_categories = d.nlargest(5, 'Profit')
    top5_profit_category = pd.concat([top5_profit_category,high_profit_categories])

NameError: name 'category_yearly_profit' is not defined

In [19]:
top5_profit_category.style.background_gradient(cmap='Spectral', subset=pd.IndexSlice[:, 'Profit'])

NameError: name 'top5_profit_category' is not defined

In [20]:
data['Unit_price'] = data['Sales']/data['Quantity']
data['Order_day'] = data['Order Date'].dt.day

NameError: name 'data' is not defined

In [21]:
g5 = sns.FacetGrid(data, row = 'Order_day', col = 'Order_year', hue = 'Order_day')
kwe = dict(s = 50, linewidth = 0.5, edgecolor = 'black')
g5 = g5.map(plt.scatter, 'Unit_price', 'Sales')
g5.set(xlim=(0,100), ylim=(0,100))
for ax in g5.axes.flat:
    ax.plot((0,100),(0,100), c = 'gray', ls = '--')
g5.add_legend()

NameError: name 'data' is not defined

In [ ]:
data['Delivery_duration'] = data['Ship Date']-data['Order Date']
country_group = data.groupby('Country')
delivery_duration_df = country_group.agg({'Delivery_duration':'mean'})
delivery_duration_df['Duration_in_hours'] = delivery_duration_df['Delivery_duration'] / dt.timedelta(hours=1)

In [22]:
delivery_duration_df

NameError: name 'delivery_duration_df' is not defined

In [23]:
top10_sales_country_DD =top_10_sales.merge(delivery_duration_df, how='left', left_index=True, right_index=True)
top10_sales_country_DD.reset_index(inplace=True)
top10_sales_country_DD.sort_values(by='Duration_in_hours')
# we can see that China and Brazil are providing fastest deliveries

NameError: name 'top_10_sales' is not defined

In [24]:
top10_sales_country_DD.iplot(kind='bar',x='Country', y='Duration_in_hours',
                            title= 'Countries And their Average Product Delivery Duration in Hours',
                            xTitle='Countries',
                            yTitle= 'AVG Delivery Duration in hours')

NameError: name 'top10_sales_country_DD' is not defined